Authors' Note: Original code is from Microsoft Recommenders and was modified to meet the objectives of the project.


<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

Argyriou, A., González-Fierro, M., & Zhang, L. (2020). Microsoft Recommenders: Best Practices for Production-Ready Recommendation Systems. Companion Proceedings of the Web Conference 2020, 50–51. https://doi.org/10.1145/3366424.3382692

### Smart Adaptive Recommendations (SAR) 

SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history. It is powered by understanding the similarity between items, and recommending similar items to those a user has an existing affinity for.

### Global Settings and Imports## 3 SAR single-node based movie recommender

In [1]:
import sys
import logging
import scipy
import numpy as np
import pandas as pd

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.sar import SAR
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")

c:\Users\angel\anaconda3\envs\recommenders_env\lib\site-packages\pandera\_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


System version: 3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]
Pandas version: 2.2.3
NumPy version: 1.26.4
SciPy version: 1.13.1


In [ ]:
# top k items to recommend
TOP_K = 10

In [3]:
# set log level to INFO
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

### Load Data


In [15]:
own_data = pd.read_csv(r'P:\pCloud Offline\PLUS\2nd Sem\Privacy Engineering\Data\Results\mondrian_k15_100_100.csv')
own_data.head()

,userID,itemID,rating,timestamp
0,0,5588,1,0
1,0,8034,1,0
2,0,9012,1,0
3,0,9014,1,0
4,0,12589,1,0


### Split the data using the python random splitter

In [ ]:
header = {
    "col_user": "userID",
    "col_item": "itemID",
    "col_rating": "rating",
    "col_timestamp": "timestamp",
    "col_prediction": "Prediction",
}

train, test = python_stratified_split(
    own_data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42
)


In [19]:
model = SAR(
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [20]:
model.fit(train)

2025-06-21 11:35:31 INFO     Collecting user affinity matrix
2025-06-21 11:35:31 INFO     Calculating time-decayed affinities
2025-06-21 11:35:31 INFO     Creating index columns
2025-06-21 11:35:31 INFO     Building user affinity sparse matrix
2025-06-21 11:35:31 INFO     Calculating item co-occurrence
2025-06-21 11:35:31 INFO     Calculating item similarity
2025-06-21 11:35:31 INFO     Using jaccard based similarity
2025-06-21 11:35:31 INFO     Done training


In [21]:
top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

2025-06-21 11:35:31 INFO     Calculating recommendation scores
2025-06-21 11:35:31 INFO     Removing seen items


### Evaluate the results


In [23]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(
    col_user="userID",
    col_item="itemID",
    col_rating="rating",
    col_prediction="Prediction",
    relevancy_method="top_k",
    k=TOP_K,
)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

In [24]:
print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		 10
MAP:		 0.932613
NDCG:		 0.956084
Precision@K:	 0.932075
Recall@K:	 0.514107
